<a href="https://colab.research.google.com/github/ruirui-zhang/RAG-few-shot-learning/blob/main/OCBang_TH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The objective is to train a LLM to label talents into categories. There are several ways:
- zero shot learning
- RAG based few show learning


Here are some definitions:
Retrieval Augmented Generation (RAG)? The phrase Retrieval Augmented Generation (RAG) comes from a recent paper by Lewis et al. from Facebook AI. The idea is to use a pre-trained language model (LM) to generate text, but to use a separate retrieval system to find relevant documents to condition the LM on.

**What is Qdrant?** Qdrant is an open-source vector search engine that allows you to search for similar vectors in a large dataset. It is built in Rust and here we'll use the Python client to interact with it. This is the Retrieval part of RAG.

**What is Few-Shot Learning?** Few-shot learning is a type of machine learning where the model is "improved" via training or fine-tuning on a small amount of data. In this case, we'll use it to fine-tune the RAG model on a small number of examples from the SQuAD dataset. This is the Augmented part of RAG.

**What is Zero-Shot Learning?** Zero-shot learning is a type of machine learning where the model is "improved" via training or fine-tuning without any dataset specific information.

**What is Fine-Tuning?** Fine-tuning is a type of machine learning where the model is "improved" via training or fine-tuning on a small amount of data. In this case, we'll use it to fine-tune the RAG model on a small number of examples from the SQuAD dataset. The LLM is what makes the Generation part of RAG.


In this notebook, I use RAG based few shot learning to fine tune Turbo-3.5.


In [ ]:
!pip install qdrant-client
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.7/176.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver

In [ ]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer
import os
from qdrant_client.http import models
from qdrant_client.http.models import PointStruct
from qdrant_client.http.models import Distance, VectorParams

In [ ]:
!pip install pandas openai tqdm tenacity scikit-learn tiktoken python-dotenv seaborn --upgrade --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 29.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
google-colab 1.0.0 requires pandas==1.5.3, but you have p

In [ ]:
import json
import os
import time

import pandas as pd
import openai
import seaborn as sns
from tenacity import retry, wait_exponential
from tqdm import tqdm
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

import warnings
warnings.filterwarnings('ignore')

tqdm.pandas()

Generating embeddings:   0%|          | 0/2338 [37:05<?, ?it/s]


## Data Preparation and Envornment Setup

In [ ]:
os.chdir('/content/')

In [ ]:
import pandas as pd

obj = pd.read_pickle(r'train_candidateinfo.pkl')

In [ ]:
qdrant_client = QdrantClient(
    url=os.getenv("QDRANT_URL"), api_key=os.getenv("QDRANT_API_KEY"), timeout=6000, prefer_grpc=True
)

collection_name = "talent_label"

# # Create the collection, run this only once
# qdrant_client.recreate_collection(
#     collection_name=collection_name,
#     vectors_config=VectorParams(size=384, distance=Distance.COSINE),
# )

True

In [ ]:
import re

with open('job.txt', 'r') as f:
    text = f.read()

def extract_stack_definitions(text):
    # Regular expression pattern to match "stack:definition" pairs
    pattern = r"\* (.*?): (.*?)(?=\n|$)"

    # Finding all matches in the text
    matches = re.findall(pattern, text, re.DOTALL)

    # Creating a dictionary from the matches
    stack_to_definition = {match[0].strip(): match[1].strip() for match in matches}

    return stack_to_definition


# Call the function with your text
dictionary = extract_stack_definitions(text)

# Printing the dictionary
for stack, definition in dictionary.items():
    print(f'"{stack}": "{definition}",')


"Backend": "Development of server-side components, APIs, and services that power web or mobile applications.",
"Frontend": "Development of user interfaces and interactive elements for web or mobile applications.",
"Fullstack": "Combination of frontend and backend development skills, capable of working on both user-facing components and server-side logic.",
"MobileAPP Dev": "The process of creating, maintaining, and enhancing applications specifically designed for use on mobile devices, utilizing a variety of programming languages and frameworks.",
"QA & Testing": "Design and implementation of testing processes to ensure software quality, functionality, and performance.",
"Game Dev": "Design and development of video games, including game mechanics, graphics, and interactive storytelling.",
"Video and Streaming": "Focused on efficiently handling video content, including processing, encoding, delivering, and streaming. It involves expertise in video codecs, streaming protocols, media fram

## Read the important fields and generate prompt, build train / val dataset

1.   Use the info from the linkedin page and the definition from the stack to build the prompt
2. Built train / val dataset with evenly distributed classification.



In [ ]:
import pandas as pd
import numpy as np

def extract_information(obj):
    data = []
    for url, info in obj.items():
        major = [edu['fieldOfStudy'] for edu in info.get('education', [])]
        skills = [skill['skillName'] for skill in info.get('skills', [])]
        summary = info.get('headline', "")
        work_experience = [(exp.get('title', ""), exp.get('description', "")) for exp in info.get('workExperiences', [])]
        stack = info.get('stack', "")

        data.append({
            'url': url,
            'major': major,
            'skills': skills,
            'summary': summary,
            'work_experience': work_experience,
            'stack': stack
        })

    return pd.DataFrame(data)

def prepare_prompt(row, dictionary = dictionary):
    prompt = f"The person's majors are {', '.join(row['major'])}. "
    prompt += f"This person's skills are {', '.join(row['skills'])}. "
    prompt += f"Summary: {row['summary']} "
    for title, desc in row['work_experience']:
        prompt += f"Work Experience: {title} - {desc} "
    prompt += f"This person does: {dictionary.get(row['stack'])}"
    return prompt

def diverse_sample(df):
    unique_stacks = df['stack'].unique()
    train_df = pd.DataFrame()
    test_df = pd.DataFrame()

    for stack in unique_stacks:
        stack_df = df[df['stack'] == stack]
        train = stack_df.sample(frac=0.7, random_state=1)
        test = stack_df.drop(train.index)

        train_df = pd.concat([train_df, train])
        test_df = pd.concat([test_df, test])

    return train_df, test_df


df = extract_information(obj)
df['prompt'] = df.apply(prepare_prompt, axis=1)
train_df, test_df = diverse_sample(df)

## Due to time constraint, I built a small snippet of train / val as the alternative at this time.

In [ ]:
def small_sample(df):
    unique_stacks = df['stack'].unique()
    small_df = pd.DataFrame()

    for stack in unique_stacks:
        stack_df = df[df['stack'] == stack]
        stack_df = stack_df.sample(n=1, random_state=1)

        small_df = pd.concat([small_df, stack_df])

    return small_df

small_df = small_sample(df)

In [ ]:
def small_sample(df):
    unique_stacks = df['stack'].unique()
    small_df = pd.DataFrame()

    for stack in unique_stacks:
        stack_df = df[df['stack'] == stack]
        stack_df = stack_df.sample(n=1, random_state=20)
        small_df = pd.concat([small_df, stack_df])

    return small_df

small_val_df = small_sample(df)

In [ ]:
small_df.index

Int64Index([   3, 2398, 1550,  203, 2595, 2496,  231, 2610,  586,  328, 1709,
             410, 2799, 2354,  506,  974, 3141, 3076, 2302, 2088,  690, 2745,
            2488, 2171,  849, 1237,  903, 1381, 1651, 1495, 1406,  996, 1093,
            1148, 1176, 1717, 2053, 3148, 1958, 3002, 1889, 3177, 1668, 1776,
            2178, 2815],
           dtype='int64')

In [ ]:
small_val_df.index


Int64Index([   5,  736, 2586,  193,   85,  111,  261, 1796,  581,  323, 1845,
             402,  523, 2557,  498,  915, 1848, 1821, 2301, 2160,  711,  720,
            3106, 2237,  827, 1710,  870, 1359, 2275, 1524, 1437,  979, 1040,
            1057, 1156, 1849, 3204, 1558, 3168, 1984, 2988, 2873, 1673, 3326,
            2910, 2806],
           dtype='int64')

In [ ]:
df.head()

,url,major,skills,summary,work_experience,stack,prompt
0,https://www.linkedin.com/in/myuanwang,[],"[C, C++, Algorithms, Java, Programming, HTML, ...",Software Engineer at Twitter,[],Frontend,The person's majors are . This person's skills...
1,https://www.linkedin.com/in/celsius273,[],"[Java, React.js, JavaScript, C++, C, XML, HTML...",Software Engineer at Google,[],Frontend,The person's majors are . This person's skills...
2,https://www.linkedin.com/in/zhangqian-liu,[],"[Java, Python, JavaScript, HTML, XML, SQL, CSS...",Software Engineer at Google,[],Simulation,The person's majors are . This person's skills...
3,https://www.linkedin.com/in/randallcarlisle,[],"[Angular 2, React.js, Salesforce.com Developme...",Software Engineer III at Google,[],Frontend,The person's majors are . This person's skills...
4,https://www.linkedin.com/in/ivanbabak,[],"[Video Playback, Web Applications, Software Ar...",Software Engineer (UI / Front End),[],Frontend,The person's majors are . This person's skills...


In [ ]:
train_df.shape

(2333, 7)

## The description is not explicit in the instruction, because of time constraint, I used the unique categories from the train data as the label candidates at this time.

In [ ]:
unique_stacks = df['stack'].unique()
print(len(unique_stacks))
print(unique_stacks)

46
['Frontend' 'Simulation' 'Firmware Engineering' 'Hardware Test'
 'Sensors & Actuators' 'Robotics' 'Optics & Display' 'Other Hardware Dev'
 'Product Management_SW_2B' 'Audio and Speech' 'AI Engineering'
 'PCB Design' 'Embedded Systems' 'IC Design' 'Power Systems'
 'Operation Analytics' 'Program Management' 'Cloud Architecture'
 'Solutions Architect' 'Support Engineering' 'Automotive Engineering'
 'AI Research' 'Localization and Mapping' 'HPC' 'Quantum Computing'
 'Backend' 'Crypto & Blockchain' 'Fullstack' 'DevOps & SRE' 'Data Science'
 'Data Engineering' 'IT' 'Product Design' 'UI/UX Research'
 'Talent Acquisition' 'MobileAPP Dev' 'Computer Graphics & XR'
 'QA & Testing' 'Natural Language Processing' 'Systems & Networking'
 'Security & Privacy' 'Computer Vision' 'Software_TBD' 'Game Dev'
 'OS Frameworks' 'Video and Streaming']


In [ ]:
category = '\n\n'.join(unique_stacks)
category

'Frontend\n\nSimulation\n\nFirmware Engineering\n\nHardware Test\n\nSensors & Actuators\n\nRobotics\n\nOptics & Display\n\nOther Hardware Dev\n\nProduct Management_SW_2B\n\nAudio and Speech\n\nAI Engineering\n\nPCB Design\n\nEmbedded Systems\n\nIC Design\n\nPower Systems\n\nOperation Analytics\n\nProgram Management\n\nCloud Architecture\n\nSolutions Architect\n\nSupport Engineering\n\nAutomotive Engineering\n\nAI Research\n\nLocalization and Mapping\n\nHPC\n\nQuantum Computing\n\nBackend\n\nCrypto & Blockchain\n\nFullstack\n\nDevOps & SRE\n\nData Science\n\nData Engineering\n\nIT\n\nProduct Design\n\nUI/UX Research\n\nTalent Acquisition\n\nMobileAPP Dev\n\nComputer Graphics & XR\n\nQA & Testing\n\nNatural Language Processing\n\nSystems & Networking\n\nSecurity & Privacy\n\nComputer Vision\n\nSoftware_TBD\n\nGame Dev\n\nOS Frameworks\n\nVideo and Streaming'

In [ ]:
len(dictionary.keys())

129

## Build the embedding for the prompt, upload to vector db for later retrieval

In [ ]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

In [ ]:
import pandas as pd
from typing import List
from tqdm import tqdm

# Assuming PointStruct and embedding_model are already defined
# and the necessary modules are imported

def generate_points_from_dataframe(df: pd.DataFrame) -> List[PointStruct]:
    batch_size = 512
    prompts = df["prompt"].tolist()
    total_batches = len(prompts) // batch_size + 1

    pbar = tqdm(total=len(prompts), desc="Generating embeddings")

    # Generate embeddings in batches to improve performance
    embeddings = []
    for i in range(total_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(prompts))
        batch = prompts[start_idx:end_idx]

        batch_embeddings = encoder.encode(batch, batch_size=batch_size)
        embeddings.extend(batch_embeddings)
        pbar.update(len(batch))

    pbar.close()

    # Convert embeddings to list of lists
    embeddings_list = [embedding.tolist() for embedding in embeddings]

    # Create a temporary DataFrame to hold the embeddings and existing DataFrame columns
    temp_df = df.copy()
    temp_df["embeddings"] = embeddings_list
    temp_df["id"] = temp_df.index

    # Generate PointStruct objects using DataFrame apply method
    points = temp_df.apply(
        lambda row: PointStruct(
            id=row["id"],
            vector=row["embeddings"],
            payload={
                "prompt": row["prompt"],
                "stack": row["stack"],
            },
        ),
        axis=1,
    ).tolist()

    return points

# You can call the function with your DataFrame as an argument
points = generate_points_from_dataframe(train_df)



Generating embeddings: 100%|██████████| 2338/2338 [00:07<00:00, 315.99it/s]


In [ ]:
operation_info = qdrant_client.upsert(
    collection_name=collection_name, wait=True, points=points
)
print(operation_info)

operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


## Retrieve similar instance as supplement to the current prompt

In [ ]:
def get_few_shot_prompt(row):

    query, row_context = row["prompt"], row["stack"]

    embeddings = list(encoder.encode([query]))
    query_embedding = embeddings[0].tolist()

    num_of_qa_to_retrieve = 5

    # Query Qdrant for similar questions
    q_results = qdrant_client.search(
        collection_name=collection_name,
        query_vector=query_embedding,
        with_payload=True,
        limit=num_of_qa_to_retrieve,
    )

    instruction = f"""You will be provided with job talent queries. \
Classify each query into a category. \
Only choose category from those following:\
{category}.\n\n"""

    def q_to_prompt(q):
        question, context = q.payload["prompt"], q.payload["stack"]
        answer = context  # Setting the answer as the stack
        return [
            {
                "role": "user",
                "content": f"""Question: {question}\n\nAnswer:"""
            },
            {"role": "assistant", "content": answer},
        ]

    rag_prompt = []

    for q in q_results[:3]:  # Adjust the number as needed
        rag_prompt += q_to_prompt(q)

    rag_prompt += [
        {
            "role": "user",
            "content": f"""Question: {query}\n\nContext: {row_context}\n\nAnswer:"""
        },
    ]

    rag_prompt = [{"role": "system", "content": instruction}] + rag_prompt
    return rag_prompt

# You can call the function passing a row of your DataFrame as an argument
prompt = get_few_shot_prompt(train_df.iloc[0])


In [ ]:
small_df["few_shot_prompt"] = small_df.apply(get_few_shot_prompt, axis=1)

In [ ]:
# Prepare the OpenAI File format i.e. JSONL from train_sample
import json

def dataframe_to_jsonl(df):
    def create_jsonl_entry(row):
        messages = row["few_shot_prompt"]
        return json.dumps({"messages": messages})

    jsonl_output = df.apply(create_jsonl_entry, axis=1)
    return "\n".join(jsonl_output)

with open("train_few_shot.jsonl", "w") as f:
    f.write(dataframe_to_jsonl(small_df))

## Start fine tune LLM with the RAG based few shot learning

In [ ]:
class OpenAIFineTuner:
    """
    Class to fine tune OpenAI models
    """
    def __init__(self, training_file_path, model_name, suffix):
        self.training_file_path = training_file_path
        self.model_name = model_name
        self.suffix = suffix
        self.file_object = None
        self.fine_tuning_job = None
        self.model_id = None

    def create_openai_file(self):
        self.file_object = openai.File.create(
            file=open(self.training_file_path, "r"),
            purpose="fine-tune",
        )

    def wait_for_file_processing(self, sleep_time=20):
        while self.file_object.status != 'processed':
            time.sleep(sleep_time)
            self.file_object.refresh()
            print("File Status: ", self.file_object.status)

    def create_fine_tuning_job(self):
        self.fine_tuning_job = openai.FineTuningJob.create(
            training_file=self.file_object["id"],
            model=self.model_name,
            suffix=self.suffix,
        )

    def wait_for_fine_tuning(self, sleep_time=20):
        while self.fine_tuning_job.status != 'succeeded':
            time.sleep(sleep_time)
            self.fine_tuning_job.refresh()
            print("Job Status: ", self.fine_tuning_job.status)

    def retrieve_fine_tuned_model(self):
        self.model_id = openai.FineTuningJob.retrieve(self.fine_tuning_job["id"]).fine_tuned_model
        return self.model_id

    def fine_tune_model(self):
        self.create_openai_file()
        self.wait_for_file_processing()
        self.create_fine_tuning_job()
        self.wait_for_fine_tuning()
        return self.retrieve_fine_tuned_model()

fine_tuner = OpenAIFineTuner(
        training_file_path="train_few_shot.jsonl",
        model_name="gpt-3.5-turbo",
        suffix="trnfewshot20231021"
    )

model_id = fine_tuner.fine_tune_model()
model_id

File Status:  processed
Job Status:  validating_files
Job Status:  queued
Job Status:  queued
Job Status:  queued
Job Status:  queued
Job Status:  queued
Job Status:  queued
Job Status:  queued
Job Status:  queued
Job Status:  queued
Job Status:  queued
Job Status:  queued
Job Status:  queued
Job Status:  queued
Job Status:  queued
Job Status:  queued
Job Status:  queued
Job Status:  queued
Job Status:  queued
Job Status:  queued
Job Status:  queued
Job Status:  queued
Job Status:  queued
Job Status:  queued
Job Status:  queued
Job Status:  queued
Job Status:  queued
Job Status:  running
Job Status:  running
Job Status:  running
Job Status:  running
Job Status:  running
Job Status:  running
Job Status:  running
Job Status:  running
Job Status:  running
Job Status:  running
Job Status:  running
Job Status:  running
Job Status:  running
Job Status:  running
Job Status:  running
Job Status:  running
Job Status:  running
Job Status:  running
Job Status:  running
Job Status:  running
Job St

'ft:gpt-3.5-turbo-0613:personal:trnfewshot20231021:8DCXUpOP'

In [ ]:
# Function with tenacity for retries
@retry(wait=wait_exponential(multiplier=1, min=2, max=6))
def api_call(messages, model):
    return openai.ChatCompletion.create(
        model=model,
        messages=messages,
        stop=["\n\n"],
        max_tokens=100,
        temperature=0.0,
    )


# Main function to answer question
def answer_question(row, prompt_func, model="gpt-3.5-turbo-0613"):
    messages = prompt_func(row)
    response = api_call(messages, model)
    return response["choices"][0]["message"]["content"]

## Validation using the fine tuned model

In [ ]:
small_val_df["ft_generated_answer_few_shot"] = small_val_df.progress_apply(answer_question, model=model_id, prompt_func=get_few_shot_prompt, axis=1)
small_val_df.to_json("val_ft_few_shot.json", orient="records", lines=True)

100%|██████████| 46/46 [00:26<00:00,  1.73it/s]


In [ ]:
small_val_df[['stack', 'ft_generated_answer_few_shot']]

,stack,ft_generated_answer_few_shot
5,Frontend,Computer Software
736,Simulation,Computer Software
2586,Firmware Engineering,Computer Software
193,Hardware Test,Semiconductors
85,Sensors & Actuators,Electrical/Electronic Manufacturing
111,Robotics,Industrial Automation
261,Optics & Display,Computer Hardware
1796,Other Hardware Dev,Computer Hardware
581,Product Management_SW_2B,Computer Software
323,Audio and Speech,Computer Software


# As shown above, the results are not falling into the constraint categories. But it is the result with only 46 records. It can be told that the general categorization is still current.

### Evaluation for hallucination and acc

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

class Evaluator:
    def __init__(self, df, categories):
        self.df = df
        self.categories = categories  # Predefined list of categories
        self.y_pred = pd.Series()  # Initialize as empty Series

    def _evaluate_single_row(self, row, answers_column):
        generated_answer = row["ft_generated_answer_few_shot"].lower()
        actual_answer = row["stack"].lower()

        if generated_answer in self.categories:
            return "✅ Correct Category"
        elif generated_answer == actual_answer:
            return "✅ Correct Answer"
        elif generated_answer not in self.categories:
            return "❌ Hallucination"
        else:
            return "❌ Wrong Answer"

    def evaluate_model(self, answers_column="generated_answer"):
        self.y_pred = self.df.apply(self._evaluate_single_row, answers_column=answers_column, axis=1)
        return self.y_pred.value_counts()

    def print_eval(self):
        print(self.evaluate_model())

evaluator = Evaluator(small_val_df, unique_stacks)

# Call the print_eval method to print the evaluation results
evaluator.print_eval()


❌ Hallucination    46
dtype: int64


## Required format without labels.

In [ ]:
import pandas as pd


columns = ["LinkedIn URL", "Labeled Stack", "Current Company", "Title", "Start Date", "End Date", "School", "Major", "Skills", "Industry", "Location"]
df = pd.DataFrame(columns=columns)

# Filling the DataFrame
for linkedin_url, details in obj.items():
    educations = details.get('educations', [])
    if educations:
        highest_education = educations[0]  # Assuming the first one is the highest education
        school = highest_education.get('schoolName', '')
        major = highest_education.get('fieldOfStudy', '')
    else:
        school = ''
        major = ''

    skills = ', '.join([skill['skillName'] for skill in details.get('skills', [])])

    experiences = details.get('workExperience', [])
    if experiences:
        recent_experience = experiences[0]  # Assuming the first one is the most recent experience
        current_company = recent_experience.get('companyName', '')
        title = recent_experience.get('title', '')
        start_date = recent_experience.get('startDateOn', '')
        end_date = recent_experience.get('endDateOn', '')
    else:
        current_company = ''
        title = ''
        start_date = ''
        end_date = ''

    industry = details.get('industryName', '')
    location = details.get('location', {}).get('displayName', '')

    df = df.append({
        "LinkedIn URL": linkedin_url,
        "Labeled Stack": "",  # You might need to clarify what this is
        "Current Company": current_company,
        "Title": title,
        "Start Date": start_date,
        "End Date": end_date,
        "School": school,
        "Major": major,
        "Skills": skills,
        "Industry": industry,
        "Location": location
    }, ignore_index=True)

# Now df contains the data you want
print(df)


                                           LinkedIn URL Labeled Stack  \
0                 https://www.linkedin.com/in/myuanwang                 
1                https://www.linkedin.com/in/celsius273                 
2             https://www.linkedin.com/in/zhangqian-liu                 
3           https://www.linkedin.com/in/randallcarlisle                 
4                 https://www.linkedin.com/in/ivanbabak                 
...                                                 ...           ...   
3331               https://www.linkedin.com/in/tammymcl                 
3332               https://www.linkedin.com/in/todavies                 
3333     https://www.linkedin.com/in/tom-madams-8751222                 
3334  https://www.linkedin.com/in/xutong-zhou-8764a5125                 
3335                https://www.linkedin.com/in/yein-jo                 

                     Current Company                             Title  \
0                            Twitter             

# Conclusion:
using few shot learning + RAG are not enough to give accurate result. Next steps can be: Use all the trainable data to re-do the process. Give the LLM steps to classify (Upper level classification followed by detailed classification), LangChain can be introduced afterwards. There are still much more possibilites. If the dataset is big enough, then we can develop model without ChatGPT API, but with open-sourced big model like LLama, it can be efficiently fine tuned as well using techniques like Lora/Qlora.